## 5:d - Primal-Dual Int Pt. - v2 ($\lambda $ Not Eliminated)

In [97]:
# Imports
import matplotlib.pyplot as plt
import numpy as np
from numpy import log
import shutil
import sys
import os.path

Given function:
\begin{aligned}
    \min_{x_1, x_2}\quad & \frac{3}{x_1 + x_2} + e^{x_1} + (x_1 - x_2 )^2 \\
    \text{s.t. }\quad & x_1\geq 0 \\
    & x_2\geq 0 \\
    & x_{1}^2 + x_{2}^{2} \leq 2\\ 
    & x_1 - x_2 \leq 1 
\end{aligned}


Barrier Function B(x): t f(x) + $\phi$ (x)

In [98]:
# Defining our function
def my_f(x):    
    val = np.exp(x[0]) + 3/(x[0] + x[1]) + (x[0] - x[1])**2
    return val

In [99]:
# Defining the constraint functions
def my_h1(x): 
    x = x.flatten()
    val = -x[0]
    return val

def my_h2(x): 
    x = x.flatten()
    val = -x[1]
    return val

def my_h3(x): 
    x = x.flatten()
    val = x[0]**2 + x[1]**2 -2
    return val

def my_h4(x):  
    x = x.flatten()
    val = x[0] - x[1] -1
    return val

### Defining First Derivatives

In [100]:
def nablas(x):
    #x = x.flatten()
    h = 1e-5  # Step size
    # Initialize an array to store the partial derivatives
    nabla_f = np.zeros_like(x)
    nabla_h1 = np.zeros_like(x)
    nabla_h2 = np.zeros_like(x)
    nabla_h3 = np.zeros_like(x)
    nabla_h4 = np.zeros_like(x)
    
    # Compute the partial derivative for each dimension of x
    for i in range(len(x)):
        # Perturb the i-th dimension by h
        x_plus_h = x.copy()
        x_plus_h[i] += h
        
        # Compute the forward difference
        nabla_f[i] = (my_f(x_plus_h) - my_f(x)) / h
        nabla_f = np.reshape(nabla_f,(2,1)) 
        
        nabla_h1[i] = (my_h1(x_plus_h) - my_h1(x)) / h
        nabla_h1 = np.reshape(nabla_h1,(2,1))
        
        nabla_h2[i] = (my_h2(x_plus_h) - my_h2(x)) / h
        nabla_h2 = np.reshape(nabla_h2,(2,1))
        
        nabla_h3[i] = (my_h3(x_plus_h) - my_h3(x)) / h
        nabla_h3 = np.reshape(nabla_h3,(2,1))
        
        nabla_h4[i] = (my_h4(x_plus_h) - my_h4(x)) / h
        nabla_h4 = np.reshape(nabla_h4,(2,1))
    return nabla_f, nabla_h1, nabla_h2, nabla_h3, nabla_h4

### Defining Second Derivatives

In [101]:
def nablas2(x):
    x = x.flatten()
    h = 1e-5  # Step size
    identity_matrix = np.eye(len(x))  # Identity matrix
    h_matrix = h * identity_matrix

    # Initialize matrices to store the second derivatives
    second_derivative_f = np.zeros((len(x), len(x)))
    second_derivative_h1 = np.zeros((len(x), len(x)))
    second_derivative_h2 = np.zeros((len(x), len(x)))
    second_derivative_h3 = np.zeros((len(x), len(x)))
    second_derivative_h4 = np.zeros((len(x), len(x)))

    for i, h_vec in enumerate(h_matrix):
        x_plus_h = x + h_vec
        
        # Compute the forward differences for each function
        nabla_f_x_plus_h, nabla_h1_x_plus_h, nabla_h2_x_plus_h, nabla_h3_x_plus_h, nabla_h4_x_plus_h = nablas(x_plus_h)
        nabla_f_x, nabla_h1_x, nabla_h2_x, nabla_h3_x, nabla_h4_x = nablas(x)
        
        # Calculate the second derivative approximation for each function
        second_derivative_f[:, i] = (nabla_f_x_plus_h.flatten() - nabla_f_x.flatten()) / h
        second_derivative_h1[:, i] = (nabla_h1_x_plus_h.flatten() - nabla_h1_x.flatten()) / h
        second_derivative_h2[:, i] = (nabla_h2_x_plus_h.flatten() - nabla_h2_x.flatten()) / h
        second_derivative_h3[:, i] = (nabla_h3_x_plus_h.flatten() - nabla_h3_x.flatten()) / h
        second_derivative_h4[:, i] = (nabla_h4_x_plus_h.flatten() - nabla_h4_x.flatten()) / h

    return second_derivative_f, second_derivative_h1, second_derivative_h2, second_derivative_h3, second_derivative_h4

### Combined Constraint Function

$$ f(x) = 
\left[\begin{array}{ccc}
f_1 (x) \\ ... \\
f_4 (x)
\end{array}\right]
\;\; \in \mathbb{R}^ {4 \times 1} $$

In [102]:
def h_x(x):
    x = x.flatten()
    hx = np.zeros((4, 1))
    hx[0] = my_h1(x) 
    hx[1] = my_h2(x)
    hx[2] = my_h3(x)
    hx[3] = my_h4(x)
    return hx    

### Combined Derivative Constraint Function

 $$ Df(x) = 
\left[\begin{array}{ccc}
\nabla f_1 (x) ^ T \\ ... \\
\nabla f_4 (x) ^ T
\end{array}\right]
\;\; \in \mathbb{R}^ {4 \times 2} $$

In [103]:
def D_h_x(x):
    x = x.flatten()
    Dhx = np.zeros((4, 2))
    nabla_f, nabla_h1, nabla_h2, nabla_h3, nabla_h4 = nablas(x)
    Dhx[0, :] = nabla_h1.T
    Dhx[1, :] = nabla_h2.T
    Dhx[2, :] = nabla_h3.T
    Dhx[3, :] = nabla_h4.T
    return Dhx

### Making the $r_{dual}$ vector

$$
r_{\text{dual}} = \nabla f_0 (x) + D f(x)^T u \; \in \mathbb{R}^{2 \times 1}
$$


In [104]:
def r_dual(x,lamb):
    x = x.flatten()
    nabla_f, nabla_h1, nabla_h2, nabla_h3, nabla_h4 = nablas(x)
    nf = nabla_f
    nf = np.reshape(nf,(2,1))
    rdual =  nf + np.dot( D_h_x(x).T, lamb )
    return rdual

### Making the $r_{cent}$ vector

$$ r_{\text {cent }} =-\operatorname{diag}(u) f (x)-(1^T) t \; \in \mathbb{R}^ {4 \times 1}  $$

In [105]:
def r_cent(x,lamb,t):
    x = x.flatten()
    diag_lamb = np.diagflat(lamb)
    ones_arr = np.ones((4, 1))
    rcent = - np.dot( diag_lamb, h_x(x) ) - (1/t)*ones_arr
    return rcent

### Combining r

$$r = \left(\begin{array}{c}
r_{\text {dual }} \\
r_{\text {cent }} 
\end{array}\right) \;\; \in \mathbb{R}^ {6 \times 1}  $$

In [106]:
def r_combined(x,lamb,t):
    x = x.flatten()
    rdual = r_dual(x,lamb)
    rcent = r_cent(x,lamb,t)
    #rcombine = np.vstack((rdual,rcent))
    # rdual = np.reshape(rdual,(2,1))
    # rcent = np.reshape(rcent,(4,1))
    rcombine  = np.concatenate((rdual, rcent), axis=0)
    return rcombine

### Making the r' matrix

$$ r' = \left[\begin{array}{ccc}
H_{\mathrm{pd}}(x) & D f(x)^T \\
-\operatorname{diag}(u) D f(x) & -\operatorname{diag}(f(x)) 
\end{array}\right] \;\; \in \mathbb{R}^ {6 \times 6}  $$

In [107]:
def Hpd_x(x,lamb):
    x = x.flatten()
    nabla2_f, nabla2_h1, nabla2_h2, nabla2_h3, nabla2_h4 = nablas2(x)
    exp1 = nabla2_f
    exp2 = lamb[0]*nabla2_h1 + lamb[1]*nabla2_h2 + lamb[2]*nabla2_h3 + lamb[3]*nabla2_h4
    exp = exp1 + exp2
    return exp

In [108]:
def row1col2(x):
    x = x.flatten()
    v = D_h_x(x)
    valr1c2 = v.T
    return valr1c2

In [109]:
def row2col1(x,lamb):
    x = x.flatten()
    valr2c1 = np.zeros((4, 2))
    diag_lamb = np.diagflat(lamb)
    Dhx = D_h_x(x)
    valr2c1 = - np.dot( diag_lamb, Dhx )
    return valr2c1

In [110]:
def row2col2(x):
    x = x.flatten()
    hx = h_x(x)
    v = - np.diagflat(hx)
    return v

In [111]:
def nabla_r_combined(x,lamb):
    x = x.flatten()
    r1c1 = Hpd_x(x,lamb)
    r1c2 = row1col2(x)
    r2c1 = row2col1(x,lamb)
    r2c2 = row2col2(x)
    r1 = np.concatenate((r1c1,r1c2),axis=1)
    r2 = np.concatenate((r2c1,r2c2),axis=1)
    nablar = np.concatenate((r1, r2), axis=0)
    return nablar    

### Backtracking Line Search

In [112]:
def Backtrack_s(y,delta_y,t):

    x = y[:2]
    x = x.flatten()
    x = np.reshape(x,(2,1))
    lamb = y[2:6]
    lamb = lamb.flatten()
    lamb = np.reshape(lamb,(4,1))
    delta_x = delta_y[:2]
    delta_x = np.reshape(delta_x,(2,1))
    delta_lamb = delta_y[2:6]
    delta_lamb = np.reshape(delta_lamb,(4,1))

    max_s = 1
    for i in range(0,4):
        if( delta_lamb[i] < 0 ):
            v = -lamb[i]/delta_lamb[i]
            if( v < 1 ):
                max_s = v

    s = 0.999*max_s
    y_new = y + s * delta_y
    x_new = y_new[:2]
    x_new = x.flatten()
    x_new = np.reshape(x,(2,1))
    lamb_new = y_new[2:6]
    lamb = lamb.flatten()
    lamb = np.reshape(lamb,(4,1))

    while ( (my_h1(x_new) > 0) & (my_h2(x_new) > 0) & (my_h3(x_new) > 0) & (my_h4(x_new) > 0) ):
        s = beta * s
        x_new = x + s * delta_x
    
    r_old = r_combined(x,lamb,t)
    r_norm_old = np.dot(r_old.flatten(),r_old.flatten())**0.5
    r_new = r_combined(x_new,lamb_new,t)
    r_norm_new = np.dot(r_new.flatten(),r_new.flatten())**0.5
    while ( r_norm_new > (1 - alp * s) * r_norm_old ):
        s = beta * s
        x_new = x + s * delta_x
        lamb_new = lamb + s * delta_lamb
        r_new = r_combined(x_new,lamb_new,t)
        r_norm_new = np.dot(r_new.flatten(),r_new.flatten())**0.5
    return s

### Primal Dual Int Pt. Method- v2 Implementation

In [113]:
# Initialising Lambda
m = 4
x_start = np.array([0.5,0.5])  # s.t. hi(x_start) < 0 
lamb = np.ones((4, 1))         # > 0
neta = - np.dot( h_x(x_start).T, lamb)
mu = 1.2
eps = 1e-5

# Parameters for Back-Tracking
alp = 0.5
beta = 0.3

iter = 0
while True:

    t = mu*m/neta
    inv_nabla_r = np.linalg.inv(nabla_r_combined(x_start,lamb)) 
    x_start = x_start.flatten()
    x_start = np.reshape(x_start,(2,1))
    y = np.concatenate((x_start,lamb), axis=0)
    y = np.reshape(y,(6,1))
    delta_y = - np.dot( inv_nabla_r, r_combined(x_start,lamb,t).flatten() )
    delta_y = np.reshape(delta_y,(6,1))
    s = Backtrack_s(y,delta_y,t)
    y = y + s * delta_y
    x_start = y[:2]
    lamb = y[2:6]
    
    neta = - np.dot( h_x(x_start).T, lamb)
    r_dualval = r_dual(x_start,lamb)
    norm_r_dual = np.dot(r_dualval.flatten(),r_dualval.flatten())**0.5
    if ( (neta <= eps) & (norm_r_dual < eps) ):
        break
    iter += 1

print("Optimal solution:", x_start)
fopt = my_f(x_start)
print("Optimal function value:", fopt)
print("Number of iterations taken to converge:", iter)

Optimal solution: [[0.64729726]
 [1.12489091]]
Optimal function value: [3.8312888]
Number of iterations taken to converge: 241
